# **Zadanie 1**

Zaimplementuj w środowisku MPI obliczanie równoległe liczby PI z wzoru Leibniz-a.

In [44]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
 

// funkcja potęgująca bo funkcja w math.h pow nie działa 

double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int i,numer_procesu,liczba_procesow;
    int tag=50;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);

    //if proces 0 to:
    if(numer_procesu == 0)
    {

      double pole = 0 ,x=0;

     //odbieranie oraz sumowanie wynikow z wszystkich procesow

      for(i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        pole +=x;
      }
      printf("Wartosc liczby pi = %1f \n", 4 *  pole);
    }

    if(numer_procesu != 0)
    {
      double x = potegowanie( -1 , numer_procesu - 1   )   / ( 2 *  numer_procesu - 1  )   ; 
      printf("Wynik procesu %d to = %f \n" , numer_procesu , x);
      //wysylanie liczby 
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 40 --allow-run-as-root a.out

Wynik procesu 7 to = 0.076923 
Wynik procesu 11 to = 0.047619 
Wynik procesu 22 to = -0.023256 
Wynik procesu 32 to = -0.015873 
Wynik procesu 35 to = 0.014493 
Wynik procesu 23 to = 0.022222 
Wynik procesu 25 to = 0.020408 
Wynik procesu 2 to = -0.333333 
Wynik procesu 15 to = 0.034483 
Wynik procesu 36 to = -0.014085 
Wynik procesu 29 to = 0.017544 
Wynik procesu 33 to = 0.015385 
Wynik procesu 18 to = -0.028571 
Wynik procesu 8 to = -0.066667 
Wynik procesu 6 to = -0.090909 
Wynik procesu 28 to = -0.018182 
Wynik procesu 20 to = -0.025641 
Wynik procesu 3 to = 0.200000 
Wynik procesu 31 to = 0.016393 
Wynik procesu 27 to = 0.018868 
Wynik procesu 24 to = -0.021277 
Wynik procesu 17 to = 0.030303 
Wynik procesu 16 to = -0.032258 
Wynik procesu 12 to = -0.043478 
Wynik procesu 34 to = -0.014925 
Wynik procesu 30 to = -0.016949 
Wynik procesu 13 to = 0.040000 
Wynik procesu 10 to = -0.052632 
Wynik procesu 39 to = 0.012987 
Wynik procesu 5 to = 0.111111 
Wynik procesu 19 to = 0.027027 

# **Zadanie 2**

Zaimplementuj w środowisku MPI wyznaczenie numerycznej wartość całki y=f(x) (postać funkcji 
wybierasz sam) w przedziale <a,b> przy pomocy N trapezów.
Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave początek a_local i koniec b_local lokalnego przedziału 
całkowania dla danego procesu oraz liczbę N_local trapezów, z których należy policzyć całkę
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [38]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
 

// moja funkcja
double funkcja(double x)
{
  return x*x;
}

int main(int argc, char **argv)
{
    //tworzenie zmiennych globalnych
    int i;
    int liczba_procesow,numer_procesu;
    int tag=50;
    
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);

    double l_procesow = liczba_procesow, n_procesu = numer_procesu;

    double granica_dolna_calki = 1,granica_gorna_calki = 4;
    double h = (granica_gorna_calki - granica_dolna_calki) / l_procesow;
    
    //if proces 0 to:
    if(numer_procesu == 0)
    {

      double calka = 0 ,s=0;
      printf("\n Wyznaczanie numerycznej wartości całki: ");

      //odbieranie i sumowanie wynikow z wszystkich procesow

      for(i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&s, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=s;
      }

      printf(" Wartosc calki = %1f\n", h * (funkcja(granica_dolna_calki)/2 + calka 
        + funkcja(granica_gorna_calki)/2));
    }

    if(numer_procesu != 0)
    {
        double xi,wynik;
        n_procesu = numer_procesu;
        xi = granica_dolna_calki + (n_procesu/l_procesow) 
            * (granica_gorna_calki - granica_dolna_calki);

        wynik = funkcja(xi);
        printf("Wynik procesu %d to = %f \n", numer_procesu,wynik);
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 10 --allow-run-as-root a.out


Wynik procesu 6 to = 7.840000 
Wynik procesu 2 to = 2.560000 
Wynik procesu 7 to = 9.610000 
Wynik procesu 1 to = 1.690000 
Wynik procesu 3 to = 3.610000 
Wynik procesu 4 to = 4.840000 
Wynik procesu 5 to = 6.250000 
Wynik procesu 8 to = 11.560000 
Wynik procesu 9 to = 13.690000 
 Wyznaczanie numerycznej wartości całki:  Wartosc calki = 21.045000


# **Zadanie 3**
Zadanie 3
Zaimplementuj program mnożenia macierzy A przez wektor B
Proces 0, to proces Root, który zarządza obliczeniami:
- inicjuje wartości macierzy A i wektora B 
- wysyła do procesów roboczych Slave fragmenty macierzy A i wektora B 
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
 

int main(int argc, char **argv)
{
    
    int i;
    int liczba_procesow,numer_procesu;
    int tag=50;
    
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    
    //if proces 0 to:
    if(numer_procesu == 0)
    {

    }

    if(numer_procesu != 0)
    {
        
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 10 --allow-run-as-root a.out